## Querying data from Arxiv API

Atom parsing code in this notebook is mostly taken from the sample code on Arxiv API [documentation](https://arxiv.org/help/api/examples/python_arXiv_parsing_example.txt).  

In [78]:
import urllib.request
import feedparser
from pyspark.sql import Row

In [40]:
def query_arxiv(search_query, start, max_results):
    base_url = 'http://export.arxiv.org/api/query?'

    query = 'search_query=%s&start=%i&max_results=%i' % (search_query,
                                                         start,
                                                         max_results)
    # perform a GET request using the base_url and query
    response = urllib.request.urlopen(base_url+query).read()

    # parse the response using feedparser
    feed = feedparser.parse(response)

    return feed

In [73]:
def read_authors(entry_authors, author_list):
    for name in (author.name for author in entry_authors):
        # maybe consider case insensitive comparing
        if name not in map(lambda t: t[1], author_list):
            author_list.append((len(author_list),name))        

In [81]:
def create_df(author_list):
    rdd = sc.parallelize(author_list)
    author_table = rdd.map(lambda x: Row(id=int(x[0]), name=x[1]))
    author_schema = sqlContext.createDataFrame(author_table)
    return author_schema
    print("", type(author_schema))

In [82]:
def main():
    # Search parameters
    search_query = 'au:%22daphne+koller%22'
    start = 0
    max_results = 5
    
    feed = query_arxiv(search_query, start, max_results)
    
    author_list = []
    paper_list = []
    
    for entry in feed.entries:
        read_authors(entry.authors, author_list)
        
    print("Author List (ID, Name):", author_list)
    author_schema = create_df(author_list) 

In [83]:
main()

Author List: [(0, 'M. Pawan Kumar'), (1, 'Daphne Koller'), (2, 'Marc Teyssier'), (3, 'Urszula Chajewska'), (4, 'Nir Friedman'), (5, 'Ron Parr')]
<class 'pyspark.sql.dataframe.DataFrame'>
